In [1]:
import numpy as np
import pandas as pd

In [29]:
df = pd.DataFrame({"index" : list(range(0,57650))})
df = pd.concat([df, pd.read_csv("Spotify_dataset.csv")], axis = "columns")
df.head()
df = df[df["index"] < 10000]

In [30]:
df.shape

(10000, 5)

In [31]:
df.isnull().sum()

index     0
artist    0
song      0
link      0
text      0
dtype: int64

In [32]:
import nltk
from nltk.corpus import stopwords

In [33]:
def remove_stopwords(text):
    stop = stopwords.words("english")
    res = ""
    for i in text.split():
        i = i.lower()
        if i not in stop:
            res += i + " "
    res.rstrip()
    return res

In [34]:
df["text"] = df["text"].apply(remove_stopwords)

In [35]:
df["artist"] = df["artist"].apply(lambda x: "".join(x.split()))

In [36]:
df["tags"] = df["artist"] + df["text"]

In [37]:
df.head()

,index,artist,song,link,text,tags
0,0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"look face, wonderful face means something spec...","ABBAlook face, wonderful face means something ..."
1,1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"take easy me, please touch gently like summer ...","ABBAtake easy me, please touch gently like sum..."
2,2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,"i'll never know go put lousy rotten show boy, ...",ABBAi'll never know go put lousy rotten show b...
3,3,ABBA,Bang,/a/abba/bang_20598415.html,making somebody happy question give take learn...,ABBAmaking somebody happy question give take l...
4,4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,making somebody happy question give take learn...,ABBAmaking somebody happy question give take l...


In [38]:
df.drop(columns = ['text'])

,index,artist,song,link,tags
0,0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"ABBAlook face, wonderful face means something ..."
1,1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"ABBAtake easy me, please touch gently like sum..."
2,2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,ABBAi'll never know go put lousy rotten show b...
3,3,ABBA,Bang,/a/abba/bang_20598415.html,ABBAmaking somebody happy question give take l...
4,4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,ABBAmaking somebody happy question give take l...
...,...,...,...,...,...
9995,9995,JustinBieber,All Bad,/j/justin+bieber/all+bad_21071754.html,"JustinBieber[verse:] another, ain't one thing ..."
9996,9996,JustinBieber,All I Want Is You,/j/justin+bieber/all+i+want+is+you_20985618.html,JustinBiebersitting alone watching snow fall l...
9997,9997,JustinBieber,As Long As You Love Me,/j/justin+bieber/as+long+as+you+love+me_210218...,"JustinBieberlong love (love me, love me) long ..."
9998,9998,JustinBieber,Baby Lady,/j/justin+bieber/baby+lady_20467491.html,JustinBieberput outfit gap like that. one matc...


In [39]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=500,stop_words='english')

In [40]:
def remove_chorus(text):
    res = ""
    elements = ["chorus", "[chorus]", "verse", "[verse]", "x", "2", "3"]
    for i in text.split():
        if i not in elements:
            res += i + " "
    res.rstrip()
    return res

In [41]:
df["tags"] = df["tags"].apply(remove_chorus)

In [42]:
vector = cv.fit_transform(df['tags']).toarray()

In [43]:
vector.shape

(10000, 500)

In [44]:
from sklearn.metrics.pairwise import cosine_similarity

In [45]:
similarity = cosine_similarity(vector)

In [46]:
similarity[0]

array([1.        , 0.        , 0.03947368, ..., 0.10650039, 0.        ,
       0.1440976 ])

In [56]:
def recommend(song):
    index = list(df[df["song"] == song]["index"])[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    print("You may also like :- ")
    for i in distances[1:6]:
        obj = df.iloc[i[0]]
        print(f"{obj.song} by {obj.artist}")

In [57]:
recommend("Mexicali Rose")

You may also like :- 
Never Make You Cry by EricClapton
Tears For You by Judds
Leaving On A Jet Plane by JohnDenver
Big Ball's In Cowtown by GeorgeStrait
My Melancholy Baby by BingCrosby
